In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=False)


In [23]:
model_ort.device

device(type='cpu')

In [8]:

inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt")

gen_tokens = model_ort.generate(**inputs)
tokenizer.batch_decode(gen_tokens)

['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']

In [9]:
tokenizer.batch_decode(model_ort.generate(**inputs, num_beams=3, num_return_sequences=3))

['<pad> beauty & health > sexual wellness > sex toys > dildos</s>',
 '<pad> beauty & health > sexual wellness > sex toys</s><pad><pad><pad>',
 '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s>']

In [3]:
%%timeit
gen_tokens = model_ort.generate(**inputs)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


457 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
gen_tokens = model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)

670 ms ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# compare with pytorch

In [12]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoModel,
    AutoTokenizer
)

In [13]:
model_pt = AutoModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2")

In [17]:
model_pt.cuda()
model_pt.eval()
tmp = 2

In [18]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

In [19]:
%%timeit
gen_tokens = model_pt.generate(**inputs)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


279 ms ± 8.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
gen_tokens = model_pt.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


325 ms ± 9.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
